# new

In [ ]:
#|default_exp _cmds.new
#|export_as_func true

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()
import repoyard._cmds.new as this_module

In [ ]:
#|top_export
from pathlib import Path
import subprocess

from repoyard import const
from repoyard.config import StorageType

In [ ]:
#|set_func_signature
def new(
    config_path: Path|None = None,
    storage_location: str|None = None,
    repo_name: str|None = None,
    from_path: Path|None = None,
    creator_hostname: str|None = None,
    initialise_git: bool = True,
):
    """
    Create a new repoyard repository.
    """
    ...

Set up testing args

In [ ]:
# Set up test environment
tests_working_dir = const.pkg_path.parent / "tmp_tests"
test_folder_path = tests_working_dir / "_cmds" / "new"
data_path = test_folder_path / ".repoyard"
!rm -rf {test_folder_path}

In [ ]:
# Args
config_path = test_folder_path / "repoyard_config" / "config.toml"
storage_location = None
repo_name = "test_repo"
from_path = None
creator_hostname = None
add_repoyard_ignore = True
initialise_git = True

In [ ]:
# Run init
from repoyard._cmds.init import init
init(config_path=config_path, data_path=data_path)

Using a non-default config path. Please set the environment variable REPOYARD_CONFIG_PATH to the given config path for repoyard to use it. 
Creating config file at: /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/repoyard_config/config.toml
Creating folder: /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/.repoyard
Creating folder: /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/.repoyard/local_store
Creating rclone config file at: /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/repoyard_config/repoyard_rclone.conf
Done!

You can modify the config at: /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/repoyard_config/config.toml
All repoyard data is stored in: /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/.repoyard


# Function body

Process args

In [ ]:
#|export
from repoyard.config import get_config
if config_path is None:
    config_path = const.DEFAULT_CONFIG_PATH
config = get_config(config_path)
    
if storage_location is None:
    storage_location = config.default_storage_location
    
if storage_location not in config.storage_locations:
    raise ValueError(f"Invalid storage location: {storage_location}. Must be one of: {', '.join(config.storage_locations)}.")
    
if repo_name is None and from_path is None:
    raise ValueError("Either --name or --from must be provided.")

if from_path is not None:
    from_path = Path(from_path).expanduser().resolve()
    
if from_path is not None and repo_name is None:
    repo_name = from_path.name
    
from repoyard._utils import get_hostname
if creator_hostname is None:
    creator_hostname = get_hostname()

Create meta file

In [ ]:
#|export
from repoyard._repos import RepoMeta
repo_meta = RepoMeta(
    name=repo_name,
    storage_location=storage_location,
    groups=[],
    creator_hostname=creator_hostname,
)
repo_meta.save(config)

Create the repo folder

In [ ]:
#|export
repo_path = repo_meta.get_local_path(config)
repo_data_path = repo_meta.get_local_repodata_path(config)
repo_path.mkdir(parents=True, exist_ok=True)

if from_path is not None:
    from_path.rename(repo_data_path)
else:
    repo_data_path.mkdir(parents=True, exist_ok=True)

Add `.repoyard_ignore`

In [ ]:
#|export
(repo_path / ".repoyard_ignore").write_text(const.DEFAULT_REPOYARD_IGNORE);

Run `git init`

In [ ]:
#|export
if initialise_git:
    subprocess.run(["git", "init"], check=True, cwd=repo_data_path)

Initialized empty Git repository in /Users/lukastk/dev/2025-11-09_00__repoyard/tmp_tests/_cmds/new/.repoyard/fake_store/01K9Y7PC1R9X6RFNFGGRXN3ZWZ__test_repo/data/.git/
